The purpose of this notebook is to try all of the denoising methods and see which one works best. Use the following for loop to test each method on every image in quadrat_photos_subset_for_debugging.

In [22]:
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Define a path to the folder that contains the photos I want to test with loops
path = 'quadrat_photos_subset_for_debugging/*'

In [13]:
for i, file in enumerate(glob.glob(path)):
    # Read in images in color
    img = cv2.imread(file)

    # Reduce the image size by half because the windows are too huge.
    img = cv2.resize(img, (0, 0), fx=.5, fy=.5)

    # Show each image in its own window.
    cv2.imshow(f'image {i}', img)

# Allows you to close all windows (individually, or close all with ctrl+w)
cv2.waitKey(0)
cv2.destroyAllWindows()

## cv2

2D custom filter

In [14]:
# Start convolution by defining a kernel - a 25x25 matrix of all ones. 
kernel = np.ones((3,3), np.float32) / 9
# Alternate option is (5,5) and divided by 25. It makes the images even more blurry.


for i, file in enumerate(glob.glob(path)):
    # Read in images in color
    img = cv2.imread(file)

    # Reduce the image size by half because the windows are too huge.
    img = cv2.resize(img, (0, 0), fx=.5, fy=.5)

    # Apply 2D custom filter
    filter_2D = cv2.filter2D(img, -1, kernel)

    # Show each image in its own window.
    cv2.imshow(f'image {i}', filter_2D)

# Allows you to close all windows (individually, or close all with ctrl+w)
cv2.waitKey(0)
cv2.destroyAllWindows()

Blur

In [15]:

for i, file in enumerate(glob.glob(path)):
    # Read in images in color
    img = cv2.imread(file)

    # Reduce the image size by half because the windows are too huge.
    img = cv2.resize(img, (0, 0), fx=.5, fy=.5)

    # Apply blur filter
    img_blur = cv2.blur(img, (3,3))

    # Show each image in its own window.
    cv2.imshow(f'image {i}', img_blur)

# Allows you to close all windows (individually, or close all with ctrl+w)
cv2.waitKey(0)
cv2.destroyAllWindows()

Gaussian Blur

In [16]:

for i, file in enumerate(glob.glob(path)):
    # Read in images in color
    img = cv2.imread(file)

    # Reduce the image size by half because the windows are too huge.
    img = cv2.resize(img, (0, 0), fx=.5, fy=.5)

    # Apply gaussian blur filter
    img_gaussian_blur = cv2.GaussianBlur(img, (3,3), 0)

    # Show each image in its own window.
    cv2.imshow(f'image {i}', img_gaussian_blur)

# Allows you to close all windows (individually, or close all with ctrl+w)
cv2.waitKey(0)
cv2.destroyAllWindows()

Median Filter (designed to blur but maintain edges)

In [17]:

for i, file in enumerate(glob.glob(path)):
    # Read in images in color
    img = cv2.imread(file)

    # Reduce the image size by half because the windows are too huge.
    img = cv2.resize(img, (0, 0), fx=.5, fy=.5)

    # Apply median filter
    median_filtered_img = cv2.medianBlur(img, 3)

    # Show each image in its own window.
    cv2.imshow(f'image {i}', median_filtered_img)

# Allows you to close all windows (individually, or close all with ctrl+w)
cv2.waitKey(0)
cv2.destroyAllWindows()

Bilateral filter (designed to blur but maintain edges - generally better than median)

In [18]:

for i, file in enumerate(glob.glob(path)):
    # Read in images in color
    img = cv2.imread(file)

    # Reduce the image size by half because the windows are too huge.
    img = cv2.resize(img, (0, 0), fx=.5, fy=.5)

    # Apply bilateral filter
    bilateral_blurred_img = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)

    # Show each image in its own window.
    cv2.imshow(f'image {i}', bilateral_blurred_img)

# Allows you to close all windows (individually, or close all with ctrl+w)
cv2.waitKey(0)
cv2.destroyAllWindows()

Non-Local means. Very smudgy, and seems to create artifacts. But seems pretty competent at isolating shapes. It appears to be showing the green and everything else is black and white. What's going on here?

In [19]:
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage import img_as_float, img_as_ubyte

In [23]:
# BEWARE THIS FOR LOOP IT RUNS FOR A VERY LONG TIME. I'm saving the results in the folder nl_means_denoising_results so I don't have to run it again.

for i, file in enumerate(glob.glob(path)):
    # Read in images in color
    img = cv2.imread(file)

    # Reduce the image size by half because the windows are too huge.
    img = cv2.resize(img, (0, 0), fx=.5, fy=.5)

    # Convert image to a float
    img = img_as_float(img)

    # estimate the noise standard deviation from the noisy image
    sigma_est = np.mean(estimate_sigma(img, channel_axis=-1))

    # Denoise the image based on the sigma that was just found
    denoised = denoise_nl_means(img, h=1.15 * sigma_est, fast_mode=True, patch_size=5, patch_distance=3)

    # Convert it back to ubyte
    denoised_ubyte = img_as_ubyte(denoised)

    plt.imsave(f'nl_means_denoising_results/image_{i}.jpg', denoised_ubyte)



